<h1 style="color: #00BFFF;">00 |</h1>

In [1]:
# 📚 Basic libraries
import pandas as pd # data manipulation
import numpy as np # numerical operations
import os # file managment
import warnings # warning messages management
import datetime # to play with dates

In [2]:
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

In [3]:
# 🎯 Specific functions
def explore_data(data): # sum & returns duplicates, NaN & empty spaces
    duplicate_rows = data.duplicated().sum()
    nan_values = data.isna().sum()
    empty_spaces = data.eq(' ').sum()
    import pandas as pd
    exploration = pd.DataFrame({"NaN": nan_values, "EmptySpaces": empty_spaces}) # New dataframe with the results
    print(f"There are {data.duplicated().sum()} duplicate rows. Also;")
    return exploration

def get_house_lifetime(row): # returns house lifetime, based on current year - (year renovated) or (year build)
    today = datetime.datetime.today().year
    if row['yr_renovated'] != 0:
        return today - row['yr_renovated']
    else:
        return today - row['yr_built']

<h2 style="color: #008080;">Data Extraction</h2>

In [4]:
file_path = os.path.join("C:/Users/apisi/01. IronData/01. GitHub/03. Projects/05_patern_pending/00_data", "datac.csv")
datac = pd.read_csv(file_path, index_col=0) # to deal with an error `Unnamed: 0` column
datac = datac.reset_index()

<h1 style="color: #00BFFF;">02 | Data cleaning</h1>

<h2 style="color: #008080;">Dealing with Duplicates</h2>

In [5]:
explore_data(datac) # sum & returns duplicates, NaN & empty spaces

There are 0 duplicate rows. Also;


,NaN,EmptySpaces
id,0,0
date,0,0
bedrooms,0,0
bathrooms,0,0
sqft_living,0,0
sqft_lot,0,0
floors,0,0
waterfront,0,0
view,0,0
condition,0,0


In [6]:
# Nothing to do here... moving on!

<h2 style="color: #008080;">Dealing with datetime formats</h2>

In [7]:
# Convert 'date' column to datetime format (it's an object after creating a new dataset)
datac['date'] = pd.to_datetime(datac['date'])

# Dates are complex. We will create 3 new columns for year, month and day.
datac['year'] = datac['date'].dt.year
datac['month'] = datac['date'].dt.month_name().str.slice(stop=3) # instead of giving them a number, we make them categoricals, and then we will encode them

<h2 style="color: #008080;">Encoding Categoricals</h2>

In [8]:
dummies = pd.get_dummies(datac['month'], prefix='month')
datac = pd.concat([datac, dummies], axis=1)

<h2 style="color: #008080;">Saving information about Seattle houses</h2>

In [10]:
# It'll have some sense later for the conclusions :)
seattle = datac[["lat","long", "zipcode"]]
seattle.to_csv("C:/Users/apisi/01. IronData/01. GitHub/03. Projects/05_patern_pending/00_data/seattle.csv")

<h2 style="color: #008080;">Dropping unnecessary features</h2>

In [12]:
datac = datac.drop('id', axis=1) # Well, obvious or NaN? I mean, it's NaN or never.
datac = datac.drop(['lat', 'long', 'zipcode', 'date', 'month'], axis=1)
# lat, long and zipcode we have them storage in seattle dataset.
# date has fulfilled it's purpose

<h2 style="color: #008080;">Life happens</h2>

In [13]:
duplicates = datac.duplicated()
print("Number of duplicated rows:", duplicates.sum())

Number of duplicated rows: 2


In [16]:
# For some reason (we tested for duplicates at the start, and repeat duplicates = datac.duplicated() in different steps), we got 2 duplicated rows after dropping some features.
# For now, we will just drop them
datac = datac.drop_duplicates()
duplicates = datac.duplicated()
print("Number of duplicated rows:", duplicates.sum())

Number of duplicated rows: 0


<h2 style="color: #008080;">Feature Engineering</h2>

In [17]:
# Apply the function to create a new column 'house_lifetime'
datac['house_lifetime'] = datac.apply(get_house_lifetime, axis=1) # returns house lifetime, based on current year S- (year renovated) or (year build)
datac = datac.drop(['yr_built', 'yr_renovated'], axis=1) # already in house lifetime

<h2 style="color: #008080;">Target coordinates</h2>

We will just kindly move our target to the right.

In [19]:
datac.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15', 'price', 'year',
       'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan',
       'month_Jul', 'month_Jun', 'month_Mar', 'month_May', 'month_Nov',
       'month_Oct', 'month_Sep', 'house_lifetime'],
      dtype='object')

In [24]:
# Kindly moving our target to the right, as best practices
datac = datac[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15', 'year',
       'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan',
       'month_Jul', 'month_Jun', 'month_Mar', 'month_May', 'month_Nov',
       'month_Oct', 'month_Sep', 'house_lifetime', 'price']]

In [25]:
# Let's see how it looks like !
datac.sample(5)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15,year,month_Apr,month_Aug,month_Dec,month_Feb,month_Jan,month_Jul,month_Jun,month_Mar,month_May,month_Nov,month_Oct,month_Sep,house_lifetime,price
15910,3,2.50,3110,6765,2.0,0,1,4,9,2550,560,2630,7626,2014,0,0,0,0,0,1,0,0,0,0,0,0,77.0,1640000
4270,5,3.00,3620,12778,1.0,0,2,5,8,1900,1720,2930,10669,2015,0,0,0,1,0,0,0,0,0,0,0,0,59.0,858000
15754,4,2.50,3150,13700,2.0,0,0,4,9,3150,0,3200,11900,2014,0,0,0,0,0,0,0,0,1,0,0,0,57.0,1260000
4228,3,2.75,1880,1793,2.0,0,0,3,7,1810,70,1800,2537,2014,0,1,0,0,0,0,0,0,0,0,0,0,22.0,268000
15413,3,1.75,1720,7202,1.0,0,0,3,7,1720,0,1720,7625,2014,0,0,1,0,0,0,0,0,0,0,0,0,28.0,404000


<h2 style="color: #008080;">Moving on to --> 03_eda</h2>

In [26]:
cleaned = datac.copy()
cleaned.to_csv("C:/Users/apisi/01. IronData/01. GitHub/03. Projects/05_patern_pending/00_data/cleaned.csv")